In [2]:
from Step2.s3_target import S3Target
import pandas as pd
from io import StringIO
import json

Step 0

In [3]:
s3_step_1 = S3Target('',
                     '',
                     '')
s3_step_2 = S3Target('',
                     '',
                     '')

In [4]:
# Download pr.data.0.Current from S3 bucket and convert it to Pandas dataframe
pr_data_tsv = s3_step_1.download_file('pr.data.0.Current')
pr_data_io = StringIO(pr_data_tsv)
pr_data_df = pd.read_csv(pr_data_io, sep='\t', header=None)

In [5]:
# Download api population data from S3 bucket and convert it to Pandas dataframe
api_pop_json = s3_step_2.download_file('usa_population.json')
api_pop_dict = json.loads(api_pop_json)
api_pop_df = pd.DataFrame(api_pop_dict["data"])

In [6]:
# Display df
api_pop_df

,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2022,2022,331097593,united-states
1,01000US,United States,2021,2021,329725481,united-states
2,01000US,United States,2020,2020,326569308,united-states
3,01000US,United States,2019,2019,324697795,united-states
4,01000US,United States,2018,2018,322903030,united-states
5,01000US,United States,2017,2017,321004407,united-states
6,01000US,United States,2016,2016,318558162,united-states
7,01000US,United States,2015,2015,316515021,united-states
8,01000US,United States,2014,2014,314107084,united-states
9,01000US,United States,2013,2013,311536594,united-states


In [7]:
# Display df
pr_data_df

,0,1,2,3,4
0,series_id,year,period,value,footnote_codes
1,PRS30006011,1995,Q01,2.6,NaN
2,PRS30006011,1995,Q02,2.1,NaN
3,PRS30006011,1995,Q03,0.9,NaN
4,PRS30006011,1995,Q04,0.1,NaN
...,...,...,...,...,...
36647,PRS88003203,2023,Q04,115.334,NaN
36648,PRS88003203,2023,Q05,115.509,NaN
36649,PRS88003203,2024,Q01,116.936,NaN
36650,PRS88003203,2024,Q02,116.544,NaN


In [8]:
# Fix pr_data_df columns
pr_data_df.columns = pr_data_df.iloc[0]
pr_data_df = pr_data_df.drop(pr_data_df.index[0]).reset_index(drop=True)

In [9]:
# Display df
pr_data_df

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN
...,...,...,...,...,...
36646,PRS88003203,2023,Q04,115.334,NaN
36647,PRS88003203,2023,Q05,115.509,NaN
36648,PRS88003203,2024,Q01,116.936,NaN
36649,PRS88003203,2024,Q02,116.544,NaN


Step 1

In [10]:
# Finding data types of columns
api_pop_df.dtypes

ID Nation      object
Nation         object
ID Year         int64
Year           object
Population      int64
Slug Nation    object
dtype: object

In [11]:
# designing schema data types
dtype_schema = {
    'ID Nation': str,
    'Nation': str,
    'ID Year': int,
    'Year': int,
    'Population': int,
    'Slug Nation': str
}

In [12]:
# Applying data types
api_pop_df = api_pop_df.astype(dtype_schema)

In [13]:
# Creating subset of the api dataset
api_mask = (api_pop_df['Year'] >= 2013) & (api_pop_df['Year'] <= 2018)
api_pop_subset = api_pop_df[api_mask]

In [14]:
# Reset index and drop old index row
api_pop_subset = api_pop_subset.reset_index(drop=True)
api_pop_subset

,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2018,2018,322903030,united-states
1,01000US,United States,2017,2017,321004407,united-states
2,01000US,United States,2016,2016,318558162,united-states
3,01000US,United States,2015,2015,316515021,united-states
4,01000US,United States,2014,2014,314107084,united-states
5,01000US,United States,2013,2013,311536594,united-states


In [15]:
# Calculate the Mean and Standard Deviation
population_mean = api_pop_subset['Population'].mean()
population_std = api_pop_subset['Population'].std()

In [16]:
# Convert NumPy floats to Python floats
population_mean = float(population_mean)
population_std = float(population_std)

In [17]:
population_mean

317437383.0

In [18]:
population_std

4257089.5415293295

Step 2

In [19]:
# Display df
pr_data_df

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN
...,...,...,...,...,...
36646,PRS88003203,2023,Q04,115.334,NaN
36647,PRS88003203,2023,Q05,115.509,NaN
36648,PRS88003203,2024,Q01,116.936,NaN
36649,PRS88003203,2024,Q02,116.544,NaN


In [20]:
# Checking for inconsistency in columns
pr_data_df.columns


Index(['series_id        ', 'year', 'period', '       value',
       'footnote_codes'],
      dtype='object', name=0)

In [21]:
# Removing whitespace
pr_data_df.columns = pr_data_df.columns.str.strip()

In [22]:
# Checking if the columns are clean
pr_data_df.columns

Index(['series_id', 'year', 'period', 'value', 'footnote_codes'], dtype='object', name=0)

In [23]:
# Checking data types of pr_data columns
pr_data_df.dtypes

0
series_id         object
year              object
period            object
value             object
footnote_codes    object
dtype: object

In [24]:
# Designing schema
dtype_schema = {
    'series_id': str,
    'year': int,
    'value': float,
}

In [25]:
# Applying schema
pr_data_df = pr_data_df.astype(dtype_schema)

In [26]:
pr_data_df.dtypes

0
series_id          object
year                int64
period             object
value             float64
footnote_codes     object
dtype: object

In [27]:
# Display df
pr_data_df

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
36646,PRS88003203,2023,Q04,115.334,NaN
36647,PRS88003203,2023,Q05,115.509,NaN
36648,PRS88003203,2024,Q01,116.936,NaN
36649,PRS88003203,2024,Q02,116.544,NaN


In [28]:
# grouping series id and year to sum values within year
pr_data_df_grouped = pr_data_df.groupby(['series_id', 'year'])['value'].sum()

In [29]:
# Check to see if it worked
pr_data_df_grouped

series_id          year
PRS30006011        1995      7.100
                   1996     -0.500
                   1997      4.400
                   1998      4.200
                   1999     -7.700
                            ...   
PRS88003203        2020    518.133
                   2021    523.634
                   2022    562.520
                   2023    577.546
                   2024    350.137
Name: value, Length: 8325, dtype: float64

In [30]:
# Reset the index
pr_data_df_grouped = pr_data_df_grouped.reset_index()

In [31]:
pr_data_df_grouped

,series_id,year,value
0,PRS30006011,1995,7.100
1,PRS30006011,1996,-0.500
2,PRS30006011,1997,4.400
3,PRS30006011,1998,4.200
4,PRS30006011,1999,-7.700
...,...,...,...
8320,PRS88003203,2020,518.133
8321,PRS88003203,2021,523.634
8322,PRS88003203,2022,562.520
8323,PRS88003203,2023,577.546


In [32]:
# Mapping max value to each series if
pr_data_df_grouped.groupby('series_id')['value'].transform('max')

0        20.500
1        20.500
2        20.500
3        20.500
4        20.500
         ...   
8320    577.546
8321    577.546
8322    577.546
8323    577.546
8324    577.546
Name: value, Length: 8325, dtype: float64

In [33]:
# Actually applying the mapping
transformed_pr_data = pr_data_df_grouped[
    pr_data_df_grouped['value']
    == pr_data_df_grouped.groupby('series_id')['value'].transform('max')
]

In [34]:
# Check to see if it worked
transformed_pr_data

,series_id,year,value
27,PRS30006011,2022,20.500
57,PRS30006012,2022,17.100
63,PRS30006013,1998,704.125
105,PRS30006021,2010,17.600
135,PRS30006022,2010,12.500
...,...,...,...
8182,PRS88003192,2002,282.800
8233,PRS88003193,2023,847.203
8262,PRS88003201,2022,37.200
8292,PRS88003202,2022,28.700


In [35]:
# Drop old index
transformed_pr_data = transformed_pr_data.reset_index(drop=True)

In [58]:
transformed_pr_data['series_id'] = transformed_pr_data['series_id'].str.strip()

In [65]:
# Check
transformed_pr_data

,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,704.125
3,PRS30006021,2010,17.600
4,PRS30006022,2010,12.500
...,...,...,...
277,PRS88003192,2002,282.800
278,PRS88003193,2023,847.203
279,PRS88003201,2022,37.200
280,PRS88003202,2022,28.700


Part 3

In [37]:
# Check to see what I'm working with again
pr_data_df

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
36646,PRS88003203,2023,Q04,115.334,NaN
36647,PRS88003203,2023,Q05,115.509,NaN
36648,PRS88003203,2024,Q01,116.936,NaN
36649,PRS88003203,2024,Q02,116.544,NaN


In [38]:
# Check to see what I'm working with again
api_pop_df

,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2022,2022,331097593,united-states
1,01000US,United States,2021,2021,329725481,united-states
2,01000US,United States,2020,2020,326569308,united-states
3,01000US,United States,2019,2019,324697795,united-states
4,01000US,United States,2018,2018,322903030,united-states
5,01000US,United States,2017,2017,321004407,united-states
6,01000US,United States,2016,2016,318558162,united-states
7,01000US,United States,2015,2015,316515021,united-states
8,01000US,United States,2014,2014,314107084,united-states
9,01000US,United States,2013,2013,311536594,united-states


In [39]:
pr_data_df.dtypes

0
series_id          object
year                int64
period             object
value             float64
footnote_codes     object
dtype: object

In [40]:
combined_df = pr_data_df[['series_id', 'year', 'period', 'value']]

In [41]:
# Joining the tables and only keeping population
combined_df = combined_df.merge(
    api_pop_df[["Year", "Population"]],
    how="left",
    left_on="year",
    right_on="Year"
).drop(columns=["Year"])

In [42]:
combined_df

,series_id,year,period,value,Population
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
36646,PRS88003203,2023,Q04,115.334,NaN
36647,PRS88003203,2023,Q05,115.509,NaN
36648,PRS88003203,2024,Q01,116.936,NaN
36649,PRS88003203,2024,Q02,116.544,NaN


In [43]:
# The Population dtype is a float, I'm replacing NaN with 0 so I can convert column to int
combined_df["Population"] = combined_df["Population"].fillna(0)
combined_df["Population"] = combined_df["Population"].astype(int)

In [44]:
# Final Check
combined_df

,series_id,year,period,value,Population
0,PRS30006011,1995,Q01,2.600,0
1,PRS30006011,1995,Q02,2.100,0
2,PRS30006011,1995,Q03,0.900,0
3,PRS30006011,1995,Q04,0.100,0
4,PRS30006011,1995,Q05,1.400,0
...,...,...,...,...,...
36646,PRS88003203,2023,Q04,115.334,0
36647,PRS88003203,2023,Q05,115.509,0
36648,PRS88003203,2024,Q01,116.936,0
36649,PRS88003203,2024,Q02,116.544,0


In [45]:
# Make a new dataset of dropped columns
combined_df_dropped = combined_df[combined_df["Population"] != 0]
combined_df_dropped = combined_df_dropped.reset_index(drop=True)

In [46]:
# If you wanted a df of only rows with pop data
combined_df_dropped

,series_id,year,period,value,Population
0,PRS30006011,2013,Q01,1.400,311536594
1,PRS30006011,2013,Q02,0.400,311536594
2,PRS30006011,2013,Q03,0.300,311536594
3,PRS30006011,2013,Q04,0.700,311536594
4,PRS30006011,2013,Q05,0.700,311536594
...,...,...,...,...,...
12250,PRS88003203,2022,Q01,110.874,331097593
12251,PRS88003203,2022,Q02,112.204,331097593
12252,PRS88003203,2022,Q03,113.182,331097593
12253,PRS88003203,2022,Q04,113.751,331097593


In [47]:
combined_df.dtypes

series_id      object
year            int64
period         object
value         float64
Population      int64
dtype: object

In [48]:
# Insight wasn't working, exact value couldn't be found. Whitespace was in series id
combined_df['series_id'] = combined_df['series_id'].str.strip()

In [60]:
combined_insight = combined_df[
    (combined_df['series_id'] == 'PRS30006032') &
    (combined_df['year'] == 2018) &
    (combined_df['period'] == 'Q01')
]

In [61]:
combined_insight

,series_id,year,period,value,Population
1165,PRS30006032,2018,Q01,0.4,322903030


Results

1.

In [55]:
print(int(population_mean))
print(population_std)

317437383
4257089.5415293295


2.

In [56]:
transformed_pr_data

,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,704.125
3,PRS30006021,2010,17.600
4,PRS30006022,2010,12.500
...,...,...,...
277,PRS88003192,2002,282.800
278,PRS88003193,2023,847.203
279,PRS88003201,2022,37.200
280,PRS88003202,2022,28.700


In [66]:
insight = transformed_pr_data[
    transformed_pr_data['series_id'].isin(['PRS30006011', 'PRS30006012'])
]

In [67]:
insight

,series_id,year,value
0,PRS30006011,2022,20.5
1,PRS30006012,2022,17.1


3.

In [68]:
combined_df

,series_id,year,period,value,Population
0,PRS30006011,1995,Q01,2.600,0
1,PRS30006011,1995,Q02,2.100,0
2,PRS30006011,1995,Q03,0.900,0
3,PRS30006011,1995,Q04,0.100,0
4,PRS30006011,1995,Q05,1.400,0
...,...,...,...,...,...
36646,PRS88003203,2023,Q04,115.334,0
36647,PRS88003203,2023,Q05,115.509,0
36648,PRS88003203,2024,Q01,116.936,0
36649,PRS88003203,2024,Q02,116.544,0


In [69]:
combined_insight = combined_df[
    (combined_df['series_id'] == 'PRS30006032') &
    (combined_df['year'] == 2018) &
    (combined_df['period'] == 'Q01')
]

In [70]:
combined_insight

,series_id,year,period,value,Population
1165,PRS30006032,2018,Q01,0.4,322903030
